***GENERATED CODE FOR customeracq PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerAcquisitionTrainData[City]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerAcquisitionTrainData[City]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'DENVER', 'max': 'DENVER', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerAcquisitionTrainD...'}, {'feature_label': 'CustomerAcquisitionTrainData[City]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CustomerAcquisitionTrainData[City]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerAcquisitionTrainData[Product_Category]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerAcquisitionTrainData[Product_Category]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Furniture', 'max': 'Technology', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerAcquisitionTrainD...'}, {'feature_label': 'CustomerAcquisitionTrainData[Product_Category]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CustomerAcquisitionTrainData[Product_Category]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerAcquisitionTrainData[Product_Sub-Category]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerAcquisitionTrainData[Product_Sub-Category]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Appliances', 'max': 'Telephones and Communication', 'missing': '0', 'distinct': '17'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerAcquisitionTrainD...'}, {'feature_label': 'CustomerAcquisitionTrainData[Product_Sub-Category]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CustomerAcquisitionTrainData[Product_Sub-Category]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerAcquisitionTrainData[Customer Segment]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerAcquisitionTrainData[Customer Segment]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Consumer', 'max': 'Small Business', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerAcquisitionTrainD...'}, {'feature_label': 'CustomerAcquisitionTrainData[Customer Segment]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CustomerAcquisitionTrainData[Customer Segment]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerAcquisitionTrainData[First Name]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerAcquisitionTrainData[First Name]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AAXXX', 'max': 'WOXXXX', 'missing': '0', 'distinct': '173'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerAcquisitionTrainD...'}, {'feature_label': 'CustomerAcquisitionTrainData[First Name]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CustomerAcquisitionTrainData[First Name]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerAcquisitionTrainData[Last Name]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerAcquisitionTrainData[Last Name]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ABXXXX', 'max': 'ZIXXXXXXX', 'missing': '0', 'distinct': '218'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerAcquisitionTrainD...'}, {'feature_label': 'CustomerAcquisitionTrainData[Last Name]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CustomerAcquisitionTrainData[Last Name]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerAcquisitionTrainData[Address]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerAcquisitionTrainData[Address]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '10000 E ALAMEDA AVE', 'max': '9871 E WALSH PL', 'missing': '0', 'distinct': '271'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerAcquisitionTrainD...'}, {'feature_label': 'CustomerAcquisitionTrainData[Address]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CustomerAcquisitionTrainData[Address]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerAcquisitionTrainData[State]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerAcquisitionTrainData[State]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'CO', 'max': 'CO', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerAcquisitionTrainD...'}, {'feature_label': 'CustomerAcquisitionTrainData[State]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CustomerAcquisitionTrainData[State]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerAcquisitionTrainData[MOSAIC HOUSEHOLD]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerAcquisitionTrainData[MOSAIC HOUSEHOLD]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'A01', 'max': 'U00', 'missing': '0', 'distinct': '50'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerAcquisitionTrainD...'}, {'feature_label': 'CustomerAcquisitionTrainData[MOSAIC HOUSEHOLD]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CustomerAcquisitionTrainData[MOSAIC HOUSEHOLD]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerAcquisitionTrainData[MOSAIC DESCRIPTION]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerAcquisitionTrainData[MOSAIC DESCRIPTION]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Aspirational Fusion', 'max': 'Young City Solos', 'missing': '0', 'distinct': '19'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerAcquisitionTrainD...'}, {'feature_label': 'CustomerAcquisitionTrainData[MOSAIC DESCRIPTION]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CustomerAcquisitionTrainData[MOSAIC DESCRIPTION]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerAcquisitionTrainData[Channel]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerAcquisitionTrainData[Channel]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Store', 'max': 'Store', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerAcquisitionTrainD...'}, {'feature_label': 'CustomerAcquisitionTrainData[Channel]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CustomerAcquisitionTrainData[Channel]')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run customeracqHooks.ipynb
try:
	#sourcePreExecutionHook()

	customeracqhdfs = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/extra/tempFile1676297298.csv', 'filename': 'tempFile1676297298.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(customeracqhdfs)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run customeracqHooks.ipynb
try:
	#transformationPreExecutionHook()

	customeracqautofe = TransformationMain.run(customeracqhdfs,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "CustomerAcquisitionTrainData[City]", "transformation_label": "String Indexer"}], "feature": "CustomerAcquisitionTrainData[City]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "DENVER", "max": "DENVER", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerAcquisitionTrainD..."}, {"transformationsData": [{"feature_label": "CustomerAcquisitionTrainData[Product_Category]", "transformation_label": "String Indexer"}], "feature": "CustomerAcquisitionTrainData[Product_Category]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Furniture", "max": "Technology", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerAcquisitionTrainD..."}, {"transformationsData": [{"feature_label": "CustomerAcquisitionTrainData[Product_Sub-Category]", "transformation_label": "String Indexer"}], "feature": "CustomerAcquisitionTrainData[Product_Sub-Category]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Appliances", "max": "Telephones and Communication", "missing": "0", "distinct": "17"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerAcquisitionTrainD..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerAcquisitionTrainData[Count]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}, "updatedLabel": "CustomerAcquisitionTrainD..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerAcquisitionTrainData[Customer ID]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1813.76", "stddev": "1000.62", "min": "3", "max": "3374", "missing": "0"}, "updatedLabel": "CustomerAcquisitionTrainD..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerAcquisitionTrainData[Store Number]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "100.43", "stddev": "0.5", "min": "100", "max": "101", "missing": "0"}, "updatedLabel": "CustomerAcquisitionTrainD..."}, {"transformationsData": [{"feature_label": "CustomerAcquisitionTrainData[Customer Segment]", "transformation_label": "String Indexer"}], "feature": "CustomerAcquisitionTrainData[Customer Segment]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Consumer", "max": "Small Business", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerAcquisitionTrainD..."}, {"transformationsData": [{"feature_label": "CustomerAcquisitionTrainData[First Name]", "transformation_label": "String Indexer"}], "feature": "CustomerAcquisitionTrainData[First Name]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AAXXX", "max": "WOXXXX", "missing": "0", "distinct": "173"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerAcquisitionTrainD..."}, {"transformationsData": [{"feature_label": "CustomerAcquisitionTrainData[Last Name]", "transformation_label": "String Indexer"}], "feature": "CustomerAcquisitionTrainData[Last Name]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ABXXXX", "max": "ZIXXXXXXX", "missing": "0", "distinct": "218"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerAcquisitionTrainD..."}, {"transformationsData": [{"feature_label": "CustomerAcquisitionTrainData[Address]", "transformation_label": "String Indexer"}], "feature": "CustomerAcquisitionTrainData[Address]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "10000 E ALAMEDA AVE", "max": "9871 E WALSH PL", "missing": "0", "distinct": "271"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerAcquisitionTrainD..."}, {"transformationsData": [{"feature_label": "CustomerAcquisitionTrainData[State]", "transformation_label": "String Indexer"}], "feature": "CustomerAcquisitionTrainData[State]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "CO", "max": "CO", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerAcquisitionTrainD..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerAcquisitionTrainData[Zip]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "80227.75", "stddev": "12.5", "min": "80203", "max": "80249", "missing": "0"}, "updatedLabel": "CustomerAcquisitionTrainD..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerAcquisitionTrainData[DriveTime]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "11.03", "stddev": "7.34", "min": "0.65", "max": "30.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CustomerAcquisitionTrainD..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerAcquisitionTrainData[Length of Residense]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "10.35", "stddev": "11.52", "min": "0", "max": "55", "missing": "0"}, "updatedLabel": "CustomerAcquisitionTrainD..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerAcquisitionTrainData[MOR BANK: UPSCALE MERCHANDISE BUYER]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "CustomerAcquisitionTrainD..."}, {"transformationsData": [{"feature_label": "CustomerAcquisitionTrainData[MOSAIC HOUSEHOLD]", "transformation_label": "String Indexer"}], "feature": "CustomerAcquisitionTrainData[MOSAIC HOUSEHOLD]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A01", "max": "U00", "missing": "0", "distinct": "50"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerAcquisitionTrainD..."}, {"transformationsData": [{"feature_label": "CustomerAcquisitionTrainData[MOSAIC DESCRIPTION]", "transformation_label": "String Indexer"}], "feature": "CustomerAcquisitionTrainData[MOSAIC DESCRIPTION]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Aspirational Fusion", "max": "Young City Solos", "missing": "0", "distinct": "19"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerAcquisitionTrainD..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerAcquisitionTrainData[Customer_Lon]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-104.97", "stddev": "0.08", "min": "-105.144003", "max": "-104.750245", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CustomerAcquisitionTrainD..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerAcquisitionTrainData[Customer_Lat]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "39.7", "stddev": "0.03", "min": "39.632031", "max": "39.798203", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CustomerAcquisitionTrainD..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerAcquisitionTrainData[Store ID]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "100.43", "stddev": "0.5", "min": "100", "max": "101", "missing": "0"}, "updatedLabel": "CustomerAcquisitionTrainD..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerAcquisitionTrainData[Store_Lon]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-104.97", "stddev": "0.07", "min": "-105.053032", "max": "-104.905387", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CustomerAcquisitionTrainD..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerAcquisitionTrainData[Store_Lat]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "39.68", "stddev": "0.02", "min": "39.65617", "max": "39.699065", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CustomerAcquisitionTrainD..."}, {"transformationsData": [{"feature_label": "CustomerAcquisitionTrainData[Channel]", "transformation_label": "String Indexer"}], "feature": "CustomerAcquisitionTrainData[Channel]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Store", "max": "Store", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerAcquisitionTrainD..."}]}))

	#transformationPostExecutionHook(customeracqautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run customeracqHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(customeracqautofe, ["CustomerAcquisitionTrainData[Count]", "CustomerAcquisitionTrainData[Customer ID]", "CustomerAcquisitionTrainData[Store Number]", "CustomerAcquisitionTrainData[Zip]", "CustomerAcquisitionTrainData[DriveTime]", "CustomerAcquisitionTrainData[Length of Residense]", "CustomerAcquisitionTrainData[MOR BANK: UPSCALE MERCHANDISE BUYER]", "CustomerAcquisitionTrainData[Customer_Lon]", "CustomerAcquisitionTrainData[Customer_Lat]", "CustomerAcquisitionTrainData[Store ID]", "CustomerAcquisitionTrainData[Store_Lon]", "CustomerAcquisitionTrainData[Store_Lat]", "CustomerAcquisitionTrainData[City]_stringindexer", "CustomerAcquisitionTrainData[Product_Category]_stringindexer", "CustomerAcquisitionTrainData[Product_Sub-Category]_stringindexer", "CustomerAcquisitionTrainData[Customer Segment]_stringindexer", "CustomerAcquisitionTrainData[First Name]_stringindexer", "CustomerAcquisitionTrainData[Last Name]_stringindexer", "CustomerAcquisitionTrainData[Address]_stringindexer", "CustomerAcquisitionTrainData[State]_stringindexer", "CustomerAcquisitionTrainData[MOSAIC HOUSEHOLD]_stringindexer", "CustomerAcquisitionTrainData[MOSAIC DESCRIPTION]_stringindexer"], "CustomerAcquisitionTrainData[Channel]_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

